In [1]:
import cv2
import mediapipe as mp
import numpy as np

# ------------------------------
# Pose Estimation Setup
# ------------------------------
mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils

# Function to measure joint angle (e.g., shoulder–elbow–wrist)
def get_joint_angle(point_a, point_b, point_c):
    """
    Calculate angle formed at point_b by point_a and point_c.
    Points are given as (x, y) in normalized coordinates.
    """
    a = np.array(point_a)
    b = np.array(point_b)
    c = np.array(point_c)

    # Angle calculation
    ab = a - b
    cb = c - b
    radians = np.arctan2(cb[1], cb[0]) - np.arctan2(ab[1], ab[0])
    angle = np.abs(radians * 180.0 / np.pi)

    # Normalize angle into [0, 180]
    if angle > 180:
        angle = 360 - angle
    return angle

# ------------------------------
# Bicep Curl Tracker Class
# ------------------------------
class CurlCounter:
    def __init__(self):
        self.count = 0
        self.position = None  # "up" or "down"

    def update(self, angle):
        """
        Update counter state based on elbow angle.
        """
        if angle > 155:
            self.position = "down"
        elif angle < 35 and self.position == "down":
            self.position = "up"
            self.count += 1

# ------------------------------
# Main Video Capture
# ------------------------------
def main():
    counter = CurlCounter()

    # Use webcam
    cap = cv2.VideoCapture(0)

    with mp_pose.Pose(min_detection_confidence=0.6,
                      min_tracking_confidence=0.6) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert to RGB for MediaPipe
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb.flags.writeable = False
            results = pose.process(rgb)

            # Back to BGR for OpenCV
            frame.flags.writeable = True

            # Pose landmark detection
            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark

                # Grab relevant joint coordinates
                shoulder = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                            lm[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
                elbow = [lm[mp_pose.PoseLandmark.LEFT_ELBOW].x,
                         lm[mp_pose.PoseLandmark.LEFT_ELBOW].y]
                wrist = [lm[mp_pose.PoseLandmark.LEFT_WRIST].x,
                         lm[mp_pose.PoseLandmark.LEFT_WRIST].y]

                # Calculate elbow angle
                angle = get_joint_angle(shoulder, elbow, wrist)

                # Update rep count
                counter.update(angle)

                # Display angle near elbow
                elbow_px = tuple(np.multiply(elbow, [frame.shape[1], frame.shape[0]]).astype(int))
                cv2.putText(frame, f"{int(angle)}°", elbow_px,
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

            # Draw landmarks
            mp_draw.draw_landmarks(frame, results.pose_landmarks,
                                   mp_pose.POSE_CONNECTIONS,
                                   mp_draw.DrawingSpec(color=(0, 200, 255), thickness=2),
                                   mp_draw.DrawingSpec(color=(255, 255, 255), thickness=2))

            # Counter display
            cv2.rectangle(frame, (10, 10), (200, 80), (50, 50, 50), -1)
            cv2.putText(frame, "CURLS", (20, 35),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, str(counter.count), (20, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

            # Show video feed
            cv2.imshow("Bicep Curl Tracker", frame)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
